# 使用 Unsloth 对 DeepSeek-R1-Distill-Qwen-1.5B 模型进行 LoRA 微调

本 Notebook 展示了如何使用 `unsloth` 库对 `deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B` 模型进行高效的 QLoRA (Low-Rank Adaptation) 微调。

整个流程包括：
1.  环境准备与库导入
2.  加载预训练模型和分词器 (Tokenizer)。
3.  在微调前，对模型进行简单的推理测试。
4.  下载和格式化训练数据集
5.  使用 `unsloth` 的 `FastLanguageModel` 来为模型添加 LoRA 适配器。
6.  配置 `SFTTrainer` 监督微调训练配置。
7.  启动训练，并观察 Loss 变化情况
8.  保存微调后的模型
9.  测试训练后的生成结果

### 1. 环境准备与库导入

首先，我们需要安装并导入所有必要的库。`transformers` 用于加载模型和分词器，`unsloth` 用于高效微调，`trl` 提供了 `SFTTrainer`，而 `datasets` 用于处理数据。

**注意**: 在运行此 Notebook 之前，请确保已安装所有依赖包：
```bash
pip install -r requirements.txt
```

In [1]:
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig, DataCollatorForSeq2Seq
from datasets import Dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


### 2. 加载预训练模型和分词器 (Tokenizer)

In [2]:
import os
from huggingface_hub import hf_hub_download,snapshot_download
from unsloth.models import FastLanguageModel

# -------------------------------
# 1. 设置缓存目录和镜像
# -------------------------------
os.environ["HF_HOME"] ="/home/xjg/.cache/huggingface/hub"  # 可自定义缓存路径
os.environ["HF_DATASETS_OFFLINE"] = "1"     # 只从本地缓存读取数据集，不访问网络
os.environ["TRANSFORMERS_OFFLINE"] = "1"    # "0" 表示在线模式；如果设置为 "1"，transformers 会只从本地缓存加载模型


# 国内 Hugging Face 官方 CDN 镜像（无需代理）
HF_ENDPOINT = "https://hf-mirror.com/"

# -------------------------------
# 2. 下载模型到本地
# -------------------------------
model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit"
filename = "model.safetensors"



local_model_dir = snapshot_download(
    repo_id=model_name,
    cache_dir=os.environ["HF_HOME"],
    use_auth_token=None,  # 如果是私有模型，需要填入 token
    endpoint=HF_ENDPOINT
)
print(f"模型下载到本地目录：{local_model_dir}")

# 定义模型和一些基本参数
max_seq_length = 8192
dtype = None # None 表示自动选择 (Float16 a T4, V100, BFloat16 a Ampere)
load_in_4bit = True # 使用 4bit 量化加载

# 这是您的模型标识符，请替换为您正在使用的模型
# 例如："qwen-1.5b_lora_model"
# model_name = "qwen-1.5b_lora_model" 
# model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B" 
# model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit"

# model_name = "unsloth/DeepSeek-R1-Distill-Qwen-14B-unsloth-bnb-4bit" 

# 这一步会返回一个经过 Unsloth 优化的模型和一个分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = local_model_dir ,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    
)

模型下载到本地目录：/home/xjg/.cache/huggingface/hub/models--unsloth--DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit/snapshots/c93b549ceae3f8137b567af565c2b204b3045091
==((====))==  Unsloth 2025.8.5: Fast Qwen2 patching. Transformers: 4.55.2.
   \\   /|    NVIDIA GeForce RTX 3060 Laptop GPU. Num GPUs = 1. Max memory: 5.799 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


### 3. 微调前推理测试

在对模型进行任何修改之前，我们先用它来生成一段文本，看看原始模型的表现如何。这可以作为我们微调效果的基准参考。

In [3]:
# 模型推理的 Prompt 模板
inference_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>{}
"""

In [4]:
question = "男，28岁，程序员，最近一周每天工作到半夜，感觉头晕、脖子疼，有时候还恶心。"
inference_prompt.format(question, "")

'以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。\n请撰写一份恰当的回复，以完成该请求。\n在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。\n\n\n### Instruction:\n你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。\n请回答以下医学问题。\n\n### Question:\n男，28岁，程序员，最近一周每天工作到半夜，感觉头晕、脖子疼，有时候还恶心。\n\n### Response:\n<think>\n'

In [5]:
FastLanguageModel.for_inference(model)

question = "男，28岁，程序员，最近一周每天工作到半夜，感觉头晕、脖子疼，有时候还恶心。"

inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")
attention_mask = inputs.input_ids.ne(tokenizer.pad_token_id).long().to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

In [6]:
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [7]:
print(response[0].split("### Response:")[1])


<think>
好，我现在需要处理这个医学问题。让我先仔细阅读一下问题描述。这位28岁的男性，是程序员，最近一周每天工作到半夜，感觉头晕、脖子疼，有时候还恶心。看起来他可能有中风的可能性，因为头晕、恶心是常见的中风症状。

首先，我需要评估他的症状。头晕和恶心通常是中风后的常见表现，但也有其他原因，比如脑损伤、外伤等。考虑到他工作时间长，可能有神经损伤的风险，所以需要谨慎判断。

接下来，我应该考虑他的其他症状，比如是否存在恶心、头晕、恶心、呕吐等。如果这些症状持续，可能需要进一步检查。同时，是否有其他症状，比如意识模糊、意识不清晰，或者有其他并发症，比如高血压、糖尿病等。

然后，我需要考虑他的年龄和工作状态。他28岁，是程序员，工作时间长，可能有神经损伤的风险，但同时也有一定精力去工作，所以可能需要更多的关注。

接下来，我需要评估他的是否存在中风的可能性。通常，中风会导致头晕、恶心、恶心、呕吐，但这些症状可能较轻，通常需要及时就医。如果他没有其他明显症状，比如意识模糊、意识不清晰，可能需要进一步检查。

然后，我需要考虑是否需要立即就医。如果他有恶心、头晕、恶心、呕吐，可能需要立即就医，但需要根据具体表现和严重程度来决定。如果症状较轻，可能需要先观察，或者进行轻度干预，但需要谨慎，因为中风会导致严重并发症。

此外，我还需要考虑他的年龄和职业。28岁是中风的高危人群，因为中风后5-10年内有25%的中风患者有复发风险，所以需要特别注意。

接下来，我需要评估是否需要进一步检查。如果他有恶心、头晕、恶心、呕吐，可能需要立即就医，但需要根据具体症状和严重程度来决定。如果症状较轻，可能需要先观察，或者进行轻度干预，但需要谨慎，因为中风会导致严重并发症。

如果他有其他症状，比如意识模糊、意识不清晰，可能需要立即就医，但需要根据具体情况来判断。如果他没有这些症状，可能需要考虑是否需要进一步检查，比如血常规、电解质、血糖等，以评估是否存在其他并发症。

此外，我还需要评估他的是否存在神经系统损伤，比如脑膜炎、脑炎等，这些可能需要进一步诊断。如果他有脑膜炎或脑炎，可能需要立即就医，但需要根据具体症状和严重程度来决定。

最后，我需要总结我的建议，包括是否需要立即就医，可能需要立即行动，如观察、就医等，同时需要根据具体情况调整建议，以确保最佳的治疗效果。

总结一下，我需

---

### 4. 下载和格式化训练数据集


医学推理数据集：https://huggingface.co/datasets/FreedomIntelligence/medical-o1-reasoning-SFT/viewer/zh

![dataset](images/dataset.png)

In [8]:
# 模型训练的 Prompt 模板
train_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>
{}
</think>
{}
"""

In [9]:
EOS_TOKEN = tokenizer.eos_token # 添加 EOS Token

def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        # 将 EOS Token 添加到样本最后
        text = train_prompt.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "zh", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Using the latest cached version of the dataset since FreedomIntelligence/medical-o1-reasoning-SFT couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'zh' at /home/xjg/.cache/huggingface/datasets/FreedomIntelligence___medical-o1-reasoning-sft/zh/0.0.0/fc2c9e8a37b38f38da6d449564a8c350b244aef4 (last modified on Wed Aug 27 21:12:09 2025).


In [9]:
dataset[0]["text"]

'以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。\n请撰写一份恰当的回复，以完成该请求。\n在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。\n\n\n### Instruction:\n你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。\n请回答以下医学问题。\n\n### Question:\n根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？\n\n### Response:\n<think>\n这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。\n\n用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。\n\n但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。\n\n等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。\n\n仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。\n\n回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？\n\n再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。\n\n看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。\n\n不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。\n\n

In [10]:
from IPython.display import display, Markdown

display(Markdown(dataset[0]["text"]))

以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？

### Response:
<think>
这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。

用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。

但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。

等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。

仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。

回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？

再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。

看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。

不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。

哦，该不会是夏季湿热，导致湿毒入侵，孩子的体质不能御，其病情发展成这样的感染？综合分析后我觉得‘蝼蛄疖’这个病名真是相当符合。
</think>
从中医的角度来看，你所描述的症状符合“蝼蛄疖”的病症。这种病症通常发生在头皮，表现为多处结节，溃破流脓，形成空洞，患处皮肤增厚且长期不愈合。湿热较重的夏季更容易导致这种病症的发展，特别是在免疫力较弱的儿童身上。建议结合中医的清热解毒、祛湿消肿的治疗方法进行处理，并配合专业的医疗建议进行详细诊断和治疗。
<｜end▁of▁sentence｜>

### 5. 使用 Unsloth 添加 LoRA 适配器

这是使用 `unsloth` 的核心步骤。我们调用 `FastLanguageModel.get_peft_model`，它会非常高效地为模型注入 LoRA 模块。

- `r`: LoRA 的秩 (rank)，是控制模型复杂度和参数量的关键超参数。
- `target_modules`: 指定要在哪些线性层（如注意力机制的 q, k, v, o 投影层）上应用 LoRA。
- `lora_alpha`: LoRA 的缩放因子，通常设置为 `r` 的两倍或与 `r` 相同。
- `use_gradient_checkpointing`: 一种节省显存的技术，对于训练大模型至关重要。

In [11]:
# 因为 `model` 对象现在是由 Unsloth 创建的，它包含了所有必需的属性
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
      "q_proj",
      "k_proj",
      "v_proj",
      "o_proj",
      "gate_proj",
      "up_proj",
      "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=1432,
    use_rslora=False,
    loftq_config=None,
)
# 检查模型结构，确认 LoRA 适配器已添加
print(model)

Unsloth 2025.8.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536, padding_idx=151654)
        (layers): ModuleList(
          (0): Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
       

### 6. 配置 SFTTrainer

`SFTTrainer` (Supervised Fine-tuning Trainer) 是一个专门用于指令微调的训练器。我们需要配置 `TrainingArguments` 来指定所有的训练参数，如批量大小、学习率、优化器等。

In [12]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 3,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 1432,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

### 7. 开始训练

一切准备就绪后，调用 `trainer.train()` 即可开始微调过程。训练结束后，会返回包含训练统计信息（如训练损失）的对象。

In [13]:
trainer_stats = trainer.train()

# 打印训练统计信息
print(trainer_stats)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 3
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 3 x 1) = 12
 "-____-"     Trainable parameters = 18,464,768 of 1,795,552,768 (1.03% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.213900
2,3.258300
3,3.197600
4,3.219500
5,3.165700
6,3.172800
7,3.023600
8,2.805000
9,2.833400
10,2.828700


TrainOutput(global_step=60, training_loss=2.486208939552307, metrics={'train_runtime': 331.5958, 'train_samples_per_second': 2.171, 'train_steps_per_second': 0.181, 'total_flos': 5165426334879744.0, 'train_loss': 2.486208939552307})


### 8. 保存微调后的模型（Lora）

训练完成后，您可以再次进行推理，比较微调后的模型与原始模型的差异。如果对结果满意，可以使用 `model.save_pretrained("your_lora_adapter_path")` 来保存训练好的 LoRA 适配器。

In [14]:
model.save_pretrained("qwen-1.5b_lora_model")

In [15]:
tokenizer.save_pretrained("qwen-1.5b_lora_model")

('qwen-1.5b_lora_model/tokenizer_config.json',
 'qwen-1.5b_lora_model/special_tokens_map.json',
 'qwen-1.5b_lora_model/chat_template.jinja',
 'qwen-1.5b_lora_model/tokenizer.json')

In [20]:
# 模型保存方式二选一（要么使用上面的分开保存，要么使用这里的合并 Lora 保存）
# model.save_pretrained_merged("qwen-1.5b_lora_model", tokenizer, save_method="merged_16bit")

### 9. 测试训练后的生成结果

In [16]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

question="一个患有急性阑尾炎的病人已经发病5天，腹痛稍有减轻但仍然发热，在体检时发现右下腹有压痛的包块，此时应如何处理？", # Question
inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1000,
    use_cache=True,
)

In [17]:
output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(output[0].split("### Response:")[1])


<think>
首先，这位病人已经发病了5天，而且还是急性阑尾炎，这种病通常会导致腹痛和发热。他现在右下腹看到压痛的包块，这个包块可能是阑尾炎的典型表现。

根据医生的诊断，这种情况应该很紧急，所以必须尽快处理。我想，应该尽快进行手术，把阑尾取出，这样能彻底解决问题。

手术是必须的，但手术需要尽快完成，不能拖延。特别是阑尾取出后，可能会有其他并发症，比如感染或者感染性腹痛，所以手术必须在最短时间内完成。

手术时间不能太长，如果太长，病人可能会有不适感，甚至影响恢复。所以，应该选择一个最快速的手术方式，比如手术室手术，这样可以尽快完成。

手术之后，病人可能会有感染，所以医生需要监测一下，确保伤口没有感染，如果出现感染，必须立即处理，避免进一步的并发症。

所以，手术应该是最紧急和必要的步骤，一旦处理到，就能彻底治愈这个病人，让他能够恢复健康。
</think>
在急性阑尾炎的情况下，右下腹出现压痛包块，必须立即采取措施。根据诊断，这种情况需要进行阑尾取出手术，以彻底解决问题。

根据医生的诊断，手术应该是最紧急和必要的步骤。手术时间不能太长，否则病人可能会有不适感，甚至影响恢复。因此，手术应该选择一个最快速的手术方式，如手术室手术，以尽快完成。

手术后，病人可能会有感染或其他并发症，因此必须立即监测，确保伤口没有感染。如果出现感染，应立即处理，以避免进一步的并发症。

因此，手术应该是最紧急的步骤，一旦处理到，就能彻底治愈病人，让他能够恢复健康。



In [18]:
def generate_response(question: str, model, tokenizer, inference_prompt: str, max_new_tokens: int = 1024) -> str:
    """
    使用指定的模型和分词器为给定的医学问题生成响应。

    Args:
        question (str): 需要模型回答的医学问题。
        model: 已加载的 Unsloth/Hugging Face 模型。
        tokenizer: 对应的分词器。
        inference_prompt (str): 用于格式化输入的 f-string 模板。
        max_new_tokens (int, optional): 生成响应的最大 token 数量。默认为 1024。

    Returns:
        str: 模型生成的响应文本，已去除 prompt 部分。
    """
    # 1. 使用模板格式化输入
    prompt = inference_prompt.format(
        question, # 填充问题
        "",       # 留空，让模型生成 CoT 和 Response
    )

    # 2. 将格式化后的 prompt 进行分词，并转移到 GPU
    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    # 3. 使用模型生成输出
    # use_cache=True 用于加速解码过程
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=max_new_tokens,
        use_cache=True,
    )
    
    # 4. 将生成的 token 解码为文本
    # skip_special_tokens=True 会移除像 EOS_TOKEN 这样的特殊标记
    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # 5. 切分字符串，只返回 "### Response:" 之后的部分
    # 使用 .split() 分割并获取响应内容，.strip() 用于去除可能存在的前后空白字符
    response_part = decoded_output.split("### Response:")
    if len(response_part) > 1:
        return response_part[1].strip()
    else:
        # 如果模型没有生成 "### Response:" 标记，则返回整个生成内容以供调试
        return decoded_output

In [19]:
my_question = "对于一名60岁男性患者，出现右侧胸疼并在X线检查中显示右侧肋膈角消失，诊断为肺结核伴右侧胸腔积液，请问哪一项实验室检查对了解胸水的性质更有帮助？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
嗯，这位60岁的男孩，他的右胸疼，X线显示肋膈角消失，这让我想到肺结核的可能。肺结核通常会导致胸腔积液，而这种积液在X线检查中可能被发现。所以，我需要考虑什么检查能帮助我们了解这种积液的性质。

首先，肺结核的急性期通常有胸腔积液，而慢性肺结核可能也有这种现象。那么，像尿酸、血常规这些检查可能不太适合，因为它们可能和肺结核的其他症状，比如胸痛和胸腔积液有关系。

再想想，血常规和尿酸检测通常用于检查肝功能，这些可能与肺结核的急性期有关，但不能直接帮助我们了解胸腔积液的性质。

然后，是血液检查。在肺结核急性期，血常规会发现血钙减少，这可能与胸腔积液有关，但同样，它可能不直接告诉我们积液的性质。

再看看，尿常规可能有帮助，因为肺结核通常会导致尿酸增加。这可能是个不错的选择，因为它可以帮助我们了解积液的酸碱平衡情况。

还有，血常规在慢性肺结核中也有用，可以帮助我们了解血钙水平，这可能与胸腔积液有关。

最后，尿常规可能比血常规更适合用来了解胸腔积液的性质，因为它能够帮助我们了解尿酸水平和血钙，从而判断积液是否酸碱平衡失衡，或者有其他相关问题。
</think>
在诊断肺结核伴右侧胸腔积液时，了解胸腔积液的性质是关键。实验室检查中，尿常规可能是一个更合适的选择，因为它能够帮助我们了解尿酸水平和血钙。如果血钙减少或血钙升高，可能与胸腔积液有关。此外，尿酸增加可能提示有酸性血钙，这可能与肺结核的急性期相关。因此，尿常规可以帮助我们更好地了解胸腔积液的性质，比如酸碱失衡或血钙变化。


In [20]:
my_question = "对于一名 28 岁的男性患者，工作是程序员，常年熬夜，最近突然感觉头晕目眩，甚至有点恶心。请问有可能是什么疾病？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
这位28岁的男性患者，工作是程序员，经常熬夜，最近感觉头晕目眩，有点恶心。这听起来有点奇怪，但也许有其他原因。考虑到他经常熬夜，可能有精神问题。头晕目眩，恶心是常见的表现，这可能和神经问题有关。也许他得了脑损伤。不过，他工作是程序员，这可能与脑损伤有关。不过，脑损伤通常和神经功能问题相关，而他经常熬夜，可能与神经问题有关。

不过，如果他经常熬夜，可能是因为脑损伤，导致睡眠困难。而他最近又感觉头晕目眩，恶心，这可能和脑损伤有关。不过，脑损伤通常和脑膜损伤有关，而这种损伤通常会导致脑功能受损，让人感觉疲劳。如果他经常熬夜，可能是因为脑损伤。

但也有其他可能，比如神经损伤，导致头晕目眩和恶心。或者，他可能得了脑膜炎，这是常见的脑损伤，导致头晕目眩和恶心。不过，脑膜炎可能不太常见，因为脑膜炎通常与脑膜病变有关，而脑膜病变通常与脑损伤有关。

再想想，可能他得了脑膜炎，这通常与脑膜病变有关，而这种病变可能导致头晕目眩和恶心。不过，脑膜炎通常与脑膜病变有关，而脑膜病变通常与脑损伤有关。不过，脑膜炎可能是一种更常见的情况。

还有一种可能，他得了脑膜炎，这通常与脑膜病变有关，导致头晕目眩和恶心。不过，脑膜炎通常与脑膜病变有关，而脑膜病变通常与脑损伤有关。不过，脑膜炎可能是一种常见的情况。

或者，他得了脑膜炎，这是一种常见脑损伤，导致头晕目眩和恶心。不过，脑膜炎通常与脑膜病变有关，而脑膜病变通常与脑损伤有关。不过，脑膜炎可能是一种更常见的情况。

不过，脑膜炎通常与脑膜病变有关，而脑膜病变通常与脑损伤有关。不过，脑膜炎可能是一种更常见的情况。

或者，他得了脑膜炎，这是一种常见脑损伤，导致头晕目眩和恶心。不过，脑膜炎通常与脑膜病变有关，而脑膜病变通常与脑损伤有关。不过，脑膜炎可能是一种更常见的情况。

或者，他得了脑膜炎，这是一种常见脑损伤，导致头晕目眩和恶心。不过，脑膜炎通常与脑膜病变有关，而脑膜病变通常与脑损伤有关。不过，脑膜炎可能是一种更常见的情况。

或者，他得了脑膜炎，这是一种常见脑损伤，导致头晕目眩和恶心。不过，脑膜炎通常与脑膜病变有关，而脑膜病变通常与脑损伤有关。不过，脑膜炎可能是一种更常见的情况。

或者，他得了脑膜炎，这是一种常见脑损伤，导致头晕目眩